# ***ADSORPTION USING AIIDALAB***


***Q1. Three types of calculations AiiDAlab offers for adsorption studies***

*Single-component adsorption isotherms* :
These are the fundamental calculations in adsorption studies and describe how much of a gas is taken up by a porous material at a constant temperature as the pressure increases. They provide direct information about adsorption capacity, the affinity of the material for the adsorbate, and the pressure range where uptake occurs. In AiiDAlab, these isotherms can be fitted to models such as Langmuir, BET, or Toth, allowing one to extract useful parameters that summarize adsorption behavior.

*Mixture adsorption via IAST (Ideal Adsorbed Solution Theory)*: 
Real-world separations typically involve mixtures of gases rather than pure components. IAST is a predictive framework that uses single-component isotherms as input to estimate how different gases will compete for adsorption sites. With this method, AiiDAlab can calculate mixture uptake and separation selectivity without the need for costly multicomponent simulations or experiments. This makes it especially useful for applications like CO₂ capture from flue gas or natural gas purification.

*Isosteric heat of adsorption*: 
This thermodynamic property measures the energy released when a gas molecule adsorbs into a porous material. AiiDAlab evaluates it by comparing isotherms collected at multiple temperatures and applying relations such as Clausius–Clapeyron. The isosteric heat reflects the strength of interaction between gas and material, and it highlights how this strength changes with loading. This information is crucial for balancing high uptake with energy-efficient regeneration in processes such as carbon capture.

***Q2. Physical properties of IRMOF-1***
job PK : 5 
Density : 0.576983 g/cm^3
ASA A^2 : 3964.65 A^2
POAV (A3) : 13737.4 A^3 - This is the total pore volume in the structure.
Porosity : 0.77498 - (porosity) tells you the relative amount compared to the total material volume. (POAV volume fraction)


***Q3. Henry coefficients***

CO2 : JOB PK 330
CH4 : JOB PK 357 (isotherm results) => 0.000001

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pyiast
import pandas as pd

df_ch4 = pd.read_csv("CH4_IRMOF1.csv")
df_co2 = pd.read_csv("CO2_IRMOF1.csv")

In [ ]:
df_ch4.head()

In [ ]:
df_co2.head()

In [ ]:
ch4_isotherm = pyiast.InterpolatorIsotherm(df_ch4,
                                    loading_key="Loading(mmol/g)",
                                    pressure_key="Pressure(bar)")

In [ ]:
co2_isotherm = pyiast.InterpolatorIsotherm(df_co2,
                                    loading_key="Loading(mmol/g)",
                                    pressure_key="Pressure(bar)")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pyiast
import pandas as pd

# Load the data (assuming your CSV files are available)
df_ch4 = pd.read_csv("CH4_IRMOF1.csv")
df_co2 = pd.read_csv("CO2_IRMOF1.csv")

# Alternative: Try using Langmuir model fit for better extrapolation
# This often works better for IAST calculations
try:
    # Fit Langmuir model to CH4 data
    ch4_langmuir = pyiast.ModelIsotherm(df_ch4,
                                        loading_key="Loading(mmol/g)",
                                        pressure_key="Pressure(bar)",
                                        model="Langmuir")
    
    # Fit Langmuir model to CO2 data  
    co2_langmuir = pyiast.ModelIsotherm(df_co2,
                                        loading_key="Loading(mmol/g)",
                                        pressure_key="Pressure(bar)",
                                        model="Langmuir")
    
    print("Using Langmuir model fits for better extrapolation")
    ch4_isotherm = ch4_langmuir
    co2_isotherm = co2_langmuir
    
except Exception as e:
    print("Langmuir fit failed, using InterpolatorIsotherm with extrapolation")
    # Create isotherms using InterpolatorIsotherm with extrapolation enabled
    ch4_isotherm = pyiast.InterpolatorIsotherm(df_ch4,
                                        loading_key="Loading(mmol/g)",
                                        pressure_key="Pressure(bar)",
                                        fill_value=df_ch4["Loading(mmol/g)"].iloc[-1])

    co2_isotherm = pyiast.InterpolatorIsotherm(df_co2,
                                        loading_key="Loading(mmol/g)",
                                        pressure_key="Pressure(bar)",
                                        fill_value=df_co2["Loading(mmol/g)"].iloc[-1])

# First, let's check the pressure ranges of our data
print("CH4 pressure range:", df_ch4['Pressure(bar)'].min(), "to", df_ch4['Pressure(bar)'].max(), "bar")
print("CO2 pressure range:", df_co2['Pressure(bar)'].min(), "to", df_co2['Pressure(bar)'].max(), "bar")

# Show all pressure points to debug
print("\nAll CH4 pressures:", sorted(df_ch4['Pressure(bar)'].tolist()))
print("All CO2 pressures:", sorted(df_co2['Pressure(bar)'].tolist()))

# Use exactly the pressure range requested: 0.1 to 4.0 with 0.2 increments
pressures = np.arange(0.1, 4.1, 0.2)
print(f"Requested pressure range: 0.1 to 4.0 bar")
print(f"Generated pressure points: {len(pressures)} points from {pressures[0]:.1f} to {pressures[-1]:.1f}")

# Define molar fractions (0.5 for both components in binary mixture)
y_ch4 = 0.5  # Molar fraction of CH4 in gas phase
y_co2 = 0.5  # Molar fraction of CO2 in gas phase

# Initialize arrays to store results
ch4_loadings = []
co2_loadings = []
successful_pressures = []

# Calculate binary mixture isotherms using IAST
for P_total in pressures:
    try:
        # Calculate partial pressures
        P_ch4 = y_ch4 * P_total  # 0.5 * P_total
        P_co2 = y_co2 * P_total  # 0.5 * P_total
        
        print(f"Trying P_total = {P_total:.1f} bar (P_CH4 = {P_ch4:.1f}, P_CO2 = {P_co2:.1f})")
        
        # Try the IAST calculation - let pyIAST handle extrapolation if needed
        # Calculate mixture loadings using IAST
        partial_pressures = [P_ch4, P_co2]
        isotherms = [ch4_isotherm, co2_isotherm]
        
        # Use pyiast.iast to calculate mixture loadings
        q_mix = pyiast.iast(partial_pressures, isotherms, verboseflag=False)
        
        ch4_loadings.append(q_mix[0])  # CH4 loading
        co2_loadings.append(q_mix[1])  # CO2 loading
        successful_pressures.append(P_total)
        print(f"  ✓ Success: CH4 = {q_mix[0]:.3f}, CO2 = {q_mix[1]:.3f}")
            
    except Exception as e:
        print(f"  ✗ Error at pressure {P_total:.1f} bar: {str(e)[:150]}...")
        continue

# Update pressures to only include successful calculations
pressures = np.array(successful_pressures)

# Convert to numpy arrays for easier handling
ch4_loadings = np.array(ch4_loadings)
co2_loadings = np.array(co2_loadings)

# Create the plot
plt.figure(figsize=(10, 6))
plt.plot(pressures, ch4_loadings, 'b-', marker='o', linewidth=2, markersize=6, label='CH₄')
plt.plot(pressures, co2_loadings, 'r-', marker='s', linewidth=2, markersize=6, label='CO₂')

plt.xlabel('Total Pressure (bar)', fontsize=12)
plt.ylabel('Loading (mmol/g)', fontsize=12)
plt.title('Binary Mixture Isotherms (CH₄/CO₂) in IRMOF-1\nMolar fractions: y_CH₄ = y_CO₂ = 0.5', fontsize=14)
plt.legend(fontsize=12)
plt.grid(True, alpha=0.3)
plt.xlim(0, 4.2)
plt.ylim(0, max(max(ch4_loadings), max(co2_loadings)) * 1.1)

# Add some styling
plt.tight_layout()
plt.show()

# Print some results for verification
print("\nBinary Mixture Isotherm Results:")
print("="*50)
print(f"{'Pressure (bar)':<15} {'CH4 Loading':<15} {'CO2 Loading':<15}")
print("="*50)
for i in range(0, len(pressures), max(1, len(pressures)//8)):  # Print ~8 points
    print(f"{pressures[i]:<15.1f} {ch4_loadings[i]:<15.3f} {co2_loadings[i]:<15.3f}")
print("="*50)
print(f"Total data points calculated: {len(pressures)}")

In [ ]:
print("\n" + "="*60)
print("SELECTIVITY CALCULATIONS")
print("="*60)

# Define specific pressures for selectivity calculation
selectivity_pressures = [0.1, 1.0, 2.0, 3.0]

print("Selectivity formula: S_CO2/CH4 = (x_CO2/x_CH4) / (y_CO2/y_CH4)")
print("Where x = mole fraction in adsorbed phase, y = mole fraction in gas phase")
print("Since y_CO2 = y_CH4 = 0.5, then S = x_CO2/x_CH4")
print()

selectivities = []
for P_target in selectivity_pressures:
    # Interpolate loadings at the exact target pressure
    q_CH4 = np.interp(P_target, pressures, ch4_loadings)
    q_CO2 = np.interp(P_target, pressures, co2_loadings)
    
    # Calculate total loading and mole fractions in adsorbed phase
    q_total = q_CH4 + q_CO2
    x_CH4 = q_CH4 / q_total
    x_CO2 = q_CO2 / q_total
    
    # Calculate selectivity
    selectivity = x_CO2 / x_CH4
    selectivities.append(selectivity)
    
    print(f"At P = {P_target:.1f} bar:")
    print(f"  CH4 loading: {q_CH4:.3f} mmol/g")
    print(f"  CO2 loading: {q_CO2:.3f} mmol/g")
    print(f"  x_CH4 (adsorbed): {x_CH4:.3f}")
    print(f"  x_CO2 (adsorbed): {x_CO2:.3f}")
    print(f"  Selectivity S_CO2/CH4: {selectivity:.2f}")
    print()

# Analysis of selectivity trend
print("SELECTIVITY ANALYSIS:")
print("="*30)
print(f"{'Pressure (bar)':<15} {'Selectivity':<12}")
print("-" * 27)
for P, S in zip(selectivity_pressures, selectivities):
    print(f"{P:<15.1f} {S:<12.2f}")

print(f"\nSelectivity trend: {selectivities[0]:.2f} → {selectivities[-1]:.2f}")
if selectivities[-1] < selectivities[0]:
    print("✓ Selectivity DECREASES with pressure")
    print("This indicates that at higher pressures, CH4 competes more effectively")
    print("for adsorption sites, reducing the preferential adsorption of CO2.")
elif selectivities[-1] > selectivities[0]:
    print("✓ Selectivity INCREASES with pressure")
    print("CO2 maintains its adsorption advantage even at higher pressures.")
else:
    print("✓ Selectivity remains relatively CONSTANT with pressure")

print(f"\nConclusion: CO2 is preferentially adsorbed over CH4 by a factor of")
print(f"{min(selectivities):.1f}-{max(selectivities):.1f}x across the pressure range studied.")
